In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
import os
import glob
import numpy as np
from matplotlib import pyplot
from PIL import Image

In [ ]:
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "/content/drive/My Drive/8536_project/CelebA/Img/img_align_celeba.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip:  
    if os.path.isdir('img_align_celeba') == 0:
    # extracting all the files 
      print('Extracting all the files now...') 
      zip.extractall() 
      print('Done!') 
    else:
      print('File has already extracted.')

In [21]:
data_path = sorted(glob.glob('img_align_celeba/*.jpg'))
# print(len(data_path))

label_path = "/content/drive/My Drive/8536_project/CelebA/Anno/list_attr_celeba.txt"
label_list = open(label_path).readlines()[2:]
data_label = []
for i in range(len(label_list)):
  data_label.append(label_list[i].split())

for m in range(len(data_label)):
  data_label[m] = [n.replace('-1', '0') for n in data_label[m]][1:]
  data_label[m] = [int(p) for p in data_label[m]]

attributes = open(label_path).readlines()[1].split()

In [22]:
import torch
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
class celeba(Dataset):
  def __init__(self, data_path=None, label_path=None):
    self.data_path = data_path
    self.label_path = label_path

    # Data transforms
    self.transform = transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
   
  def __len__(self):
    return len(self.data_path)
  
  def __getitem__(self, idx):
    image_set = Image.open(self.data_path[idx])
    image_tensor = self.transform(image_set)
    image_label = torch.Tensor(self.label_path[idx])

    return image_tensor, image_label


In [ ]:
dataset = celeba(data_path, data_label)
# split data into train, valid, test set 7:2:1
indices = list(range(202599))
split_train = 141819
split_valid = 182339
train_idx, valid_idx, test_idx = indices[:split_train], indices[split_train:split_valid], indices[split_valid:]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=train_sampler)

validloader = torch.utils.data.DataLoader(dataset, sampler=valid_sampler)

testloader =  torch.utils.data.DataLoader(dataset, sampler=test_sampler)

print(len(trainloader))
print(len(validloader))
print(len(testloader))


In [24]:
def train(model, epochs, train_all_losses, train_all_acc):
    model.train()
    # initial the running loss
    running_loss = 0.0
    # pick each data from trainloader i: batch index/ data: inputs and labels
    correct = 0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = torch.Tensor(labels)
        # print(type(labels))
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        # print statistics
        running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update parameters
        optimizer.step()
        
        result = outputs > 0.5
        correct += (result == labels).sum().item() 

        if i % 64 == 0: 
            print('Training set: [Epoch: %d, Data: %6d] Loss: %.3f' %
                  (epochs + 1, i * 64, loss.item()))
 
    acc = correct / (split_train * 40)
    running_loss /= len(trainloader)
    train_all_losses.append(running_loss)
    train_all_acc.append(acc)
    print('\nTraining set: Epoch: %d, Accuracy: %.2f %%' % (epochs + 1, 100. * acc))


In [25]:
def validation(model, val_all_losses, val_all_acc, best_acc):
    model.eval()
    validation_loss = 0.0
    correct = 0
    for data, target in validloader:
        data = data.to('cuda')
        target = target.to('cuda')
        output = model(data)

        validation_loss += criterion(output, target).item()

        result = output > 0.5
        correct += (result == target).sum().item()


    validation_loss /= len(validloader)
    acc = correct / (len(validloader) * 40)

    val_all_losses.append(validation_loss)
    val_all_acc.append(acc)

    
    print('\nValidation set: Average loss: {:.3f}, Accuracy: {:.2f}%)\n'
          .format(validation_loss, 100. * acc))
    
    return acc

In [26]:
def test(model, attr_acc, attr_name=attributes):
    test_loss = 0
    correct = 0
    pred = []
    for data, target in testloader:
        data = data.to('cuda')
        target = target.to('cuda')
        output = model(data)
        test_loss += criterion(output, target).item()

        result = output > 0.5
        correct += (result == target).sum().item()
        compare = (result == target)
        pred.append(compare[0])


    test_loss /= len(testloader)
    acc = correct / (len(testloader) * 40)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(
        test_loss, 100. * acc))
    
    for m in range(len(attr_name)):
        num = 0
        for n in range(len(pred)):
            if pred[n][m]:
                num += 1
        accuracy = num / len(pred)
        attr_acc.append(accuracy)

    for i in range(len(attr_acc)):
        print('Attribute: %s, Accuracy: %.3f' % (attr_name[i], attr_acc[i]))


In [27]:
# mish activation function
class mish(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x * (torch.tanh(F.softplus(x)))



def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        mish()
    )


def conv_dw(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
        nn.BatchNorm2d(inp),
        mish(),
    
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        mish()
    )


class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()

        self.features = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )

        self.fc = nn.Linear(1024, 40)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x


In [28]:
# define empty list to store the losses and accuracy for ploting
train_all_losses2 = []
train_all_acc2 = []
val_all_losses2 = []
val_all_acc2 = []
test_all_losses2 = 0.0
# define the training epoches
epochs = 100

In [29]:
# instantiate Net class
mobilenet = MobileNet()
# use cuda to train the network
mobilenet.to('cuda')
#loss function and optimizer
criterion = nn.BCELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(mobilenet.parameters(), lr=learning_rate, betas=(0.9, 0.999))

In [ ]:
!pip install memory-profiler
%load_ext memory_profiler

In [ ]:
%%timeit
best_acc = 0.0
for epoch in range(epochs):
    train(mobilenet, epoch, train_all_losses2, train_all_acc2)
    acc = validation(mobilenet, val_all_losses2, val_all_acc2, best_acc)
    if acc > best_acc:
      checkpoint_path = './model_checkpoint.pth'
      best_acc = acc
      # save the model and optimizer
      torch.save({'model_state_dict': mobilenet.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
      print('new best model saved')
    print("========================================================================")

In [ ]:
checkpoint_path = '/content/drive/My Drive/8536_project/model_checkpoint.pth'
model = MobileNet().to('cuda')
checkpoint = torch.load(checkpoint_path)
print("model load successfully.")

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()
attr_acc = []
test(model, attr_acc=attr_acc)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 10))
plt.barh(range(40), [100 * acc for acc in attr_acc], tick_label = attributes, fc = 'brown')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss')
plt.grid(True, linestyle='-.')
plt.plot(train_all_losses2, c='salmon', label = 'Training Loss')
plt.plot(val_all_losses2, c='brown', label = 'Validation Loss')
plt.legend(fontsize='12', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.grid(True, linestyle='-.')
plt.plot(train_all_acc2, c='salmon', label = 'Training Accuracy')
plt.plot(val_all_acc2, c='brown', label = 'Validation Accuracy')
plt.legend(fontsize='12', loc='lower right')
plt.show()

In [ ]:
# test images on the Model
transform = transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
         
image_set = Image.open('/content/ourface1.png')

image_tensor = transform(image_set)
image = torch.unsqueeze(image_tensor, 0)
image = image.to('cuda')
output = model(image)
result = output > 0.5
result = result.cpu().numpy()


for t in range(len(attributes)):
    if result[0][t] == True:
       print("Attribute: \033[1;35m%s \033[0m, \033[1;35m%s \033[0m" % (attributes[t], result[0][t]))
    else:
       print("Attribute: %s, %s" % (attributes[t], result[0][t]))